# Import Packages

In [1]:
import pandas as pd
import numpy as np 
import csv
import re
import string
import emoji

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
#demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
path = '/home/manikya_varshney/Documents/Python/Yale/final_h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [4]:
data.shape

(6737, 3)

In [5]:
data

,id,user_id,extended_tweet_full_text
0,1.304786e+18,9.053900e+07,woke up to see if the justin bieber pandemic w...
1,1.304786e+18,8.497239e+17,@TeresaCCarter2 “Our intention is to make sure...
2,1.304786e+18,1.293830e+18,For More Information contact us. \nMail:- digi...
3,1.304786e+18,1.188902e+18,"UAE reports 1,007 new Covid-19 cases, highest ..."
4,1.304786e+18,2.273830e+08,Trump officials interfered with CDC reports on...
...,...,...,...
6732,1.304427e+18,6.874206e+07,Why did Twitter suddenly reinstate @clif_high?...
6733,1.304671e+18,8.323244e+17,Denna veckas COVID-19 veckorapport från Folkhä...
6734,1.304768e+18,4.446656e+09,Republicans Defend Trump After He Admitted Dow...
6735,1.301853e+18,3.914277e+08,The recession on the back of the Government's ...


In [6]:
data['extended_tweet_full_text']

0       woke up to see if the justin bieber pandemic w...
1       @TeresaCCarter2 “Our intention is to make sure...
2       For More Information contact us. \nMail:- digi...
3       UAE reports 1,007 new Covid-19 cases, highest ...
4       Trump officials interfered with CDC reports on...
                              ...                        
6732    Why did Twitter suddenly reinstate @clif_high?...
6733    Denna veckas COVID-19 veckorapport från Folkhä...
6734    Republicans Defend Trump After He Admitted Dow...
6735    The recession on the back of the Government's ...
6736    65 Catholics across the street only 7 wearing ...
Name: extended_tweet_full_text, Length: 6737, dtype: object

#####  1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [7]:
data['extended_tweet_full_text_duplicate'] = data['extended_tweet_full_text']

In [8]:
#Convert to lower case
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.lower()

In [9]:
#Remove URLs
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "")
#data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.replace(r"https?:\/\/\S* | www\S+ | \S+\.com\S+", "")


In [10]:
#Remove user @
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.replace(r'\@[\w]+', "")

In [11]:
data['extended_tweet_full_text'][7]

"as we open up shops and transport, the cases are expected to shoot up. fear about a probable 2nd wave is far-fetched because the 1st wave itself is still strong and growing. so, here are some measures i'd highly recommend to everyone to be able to sail through this tide: (3/n)"

In [12]:
#Remove punctuations
data['extended_tweet_full_text'] = data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

In [13]:
data['extended_tweet_full_text'][7]

'as we open up shops and transport the cases are expected to shoot up fear about a probable 2nd wave is farfetched because the 1st wave itself is still strong and growing so here are some measures id highly recommend to everyone to be able to sail through this tide 3n'

In [ ]:
#More Cleaning
data['extended_tweet_full_text']=data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '').str.replace('\n',' ').str.replace('—',' ').str.strip('“').str.strip('“').str.strip('’').str.lstrip(' ').str.rstrip(' ')

In [ ]:
#Removing Emojis
#data['extended_tweet_full_text']=data['extended_tweet_full_text'].astype(str).apply(lambda x: [demoji.replace(y, '') for y in x])
#data['extended_tweet_full_text']=data['extended_tweet_full_text'].astype(str).apply(lambda x: [re.sub(emoji.get_emoji_regexp(), r"", y) for y in x])

In [ ]:
#data['extended_tweet_full_text'] = data['extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

In [ ]:
#Tokenizing
data.extended_tweet_full_text = data.extended_tweet_full_text.astype(str)
data['tokenized_extended_tweet_full_text'] = data.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)

# remove stopwords
data['filtered_extended_tweet_full_text'] = data['tokenized_extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
#Stemming
ps = PorterStemmer()
data['stemmed_extended_tweet_full_text'] = data['filtered_extended_tweet_full_text'].apply(lambda x: [ps.stem(y) for y in x])

In [ ]:
data

In [ ]:
data

In [ ]:
#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
#Lemmatizing
lemmatizer = WordNetLemmatizer()
data['lemmatized_extended_tweet_full_text'] = data['filtered_extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

In [ ]:
data

In [ ]:
#Joining the lemmetized tokens to form string
data['final'] = data['lemmatized_extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

In [ ]:
data

In [ ]:
cols = [0, 1, 2,3, 8]
data_final = data[data.columns.values[cols]]
data_final.to_csv('final_processed_h01-20200912-101538.csv',index=False)

In [ ]:
data_final

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
temp=' '.join(data['final'].tolist())
wordcloud = WordCloud(width = 800, height = 500, background_color ='white', min_font_size = 10).generate(temp)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0) 
plt.show()